In [4]:
import joblib, pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load model and dataset
model = joblib.load("decision_tree_expenses.pkl")
df = pd.read_csv("expenses_preprocessed.csv")

# Prepare test data
X_test = df[df["split"] == "test"][["description","merchant","amount","month","day","is_weekend","payment_method"]].copy()
X_test["text"] = (X_test["description"].fillna("") + " " + X_test["merchant"].fillna("")).astype(str)
X_test = X_test[["text","amount","month","day","is_weekend","payment_method"]]
y_test = df[df["split"] == "test"]["category"]

# Make predictions
preds = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, preds))
print("\nClassification Report:\n", classification_report(y_test, preds))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, preds))

Accuracy: 0.92

Classification Report:
                 precision    recall  f1-score   support

 Entertainment       1.00      0.75      0.86         4
          Food       1.00      1.00      1.00         7
        Health       1.00      0.75      0.86         4
      Shopping       0.86      1.00      0.92         6
Transportation       0.67      1.00      0.80         2
     Utilities       1.00      1.00      1.00         2

      accuracy                           0.92        25
     macro avg       0.92      0.92      0.91        25
  weighted avg       0.94      0.92      0.92        25


Confusion Matrix:
 [[3 0 0 0 1 0]
 [0 7 0 0 0 0]
 [0 0 3 1 0 0]
 [0 0 0 6 0 0]
 [0 0 0 0 2 0]
 [0 0 0 0 0 2]]


In [3]:
# Rebuild Decision Tree pipeline from the CSV and save a fresh model
import pandas as pd, joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1) Load your Phase 2 dataset (already uploaded)
df = pd.read_csv("expenses_preprocessed.csv")

# 2) Prepare features
df["text"] = (df["description"].fillna("") + " " + df["merchant"].fillna("")).astype(str)
num = ["amount","month","day","is_weekend"]
cat = ["payment_method"]

Xtr = df[df["split"].isin(["train","val"])][["text"]+num+cat]
ytr = df[df["split"].isin(["train","val"])]["category"]
Xte = df[df["split"]=="test"][["text"]+num+cat]
yte = df[df["split"]=="test"]["category"]

# 3) Same pipeline as before (serializable, no lambdas)
prep = ColumnTransformer([
    ("text", TfidfVectorizer(max_features=1500, ngram_range=(1,1)), "text"),
    ("num",  StandardScaler(with_mean=False), num),
    ("cat",  OneHotEncoder(handle_unknown="ignore"), cat),
])

pipe = Pipeline([
    ("prep", prep),
    ("clf", DecisionTreeClassifier(max_depth=12, min_samples_leaf=5, random_state=42)),
])

pipe.fit(Xtr, ytr)

# Quick eval
pred = pipe.predict(Xte)
print("Accuracy:", round(accuracy_score(yte, pred), 3))
print(classification_report(yte, pred))

# 4) Save the model so later you can just load it
joblib.dump(pipe, "decision_tree_expenses.pkl")
print("Saved:", "decision_tree_expenses.pkl")

Accuracy: 0.92
                precision    recall  f1-score   support

 Entertainment       1.00      0.75      0.86         4
          Food       1.00      1.00      1.00         7
        Health       1.00      0.75      0.86         4
      Shopping       0.86      1.00      0.92         6
Transportation       0.67      1.00      0.80         2
     Utilities       1.00      1.00      1.00         2

      accuracy                           0.92        25
     macro avg       0.92      0.92      0.91        25
  weighted avg       0.94      0.92      0.92        25

Saved: decision_tree_expenses.pkl
